https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download


# Read Data

In [199]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [282]:
import os
import pandas as pd

# Define the folder path
folder_path = "./data/kaggle_movies"

# Read all CSV files in the folder
dataframes = {}

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        df_name = file.replace(".csv", "")  # Remove .csv extension for variable name
        file_path = os.path.join(folder_path, file)
        dataframes[df_name] = pd.read_csv(file_path)

# Now you can access each dataframe using its filename as the key in the dictionary
print(dataframes.keys())  # Print all dataframe names loaded


C:\Users\JaeHoBahng\AppData\Local\Temp\ipykernel_8532\3188001465.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[df_name] = pd.read_csv(file_path)


dict_keys(['credits', 'keywords', 'links', 'links_small', 'movies_metadata', 'ratings', 'ratings_small'])


In [283]:
credits = dataframes['credits']
keywords = dataframes['keywords']
links = dataframes['links']
links_small = dataframes['links_small']
movies_metadata = dataframes['movies_metadata']
ratings = dataframes['ratings']
ratings_small = dataframes['ratings_small']

In [297]:
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')

In [298]:
merged_df = movies_metadata.merge(links, left_on='id', right_on='tmdbId', how='left')
merged_df = merged_df[~merged_df['id'].isnull()]

In [273]:
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [299]:
# import ast

# # Convert string representation of list to actual list
# keywords["keywords"] = keywords["keywords"].apply(ast.literal_eval)

# # Extract only the 'name' values
# keywords["keywords_list"] = keywords["keywords"].apply(lambda x: [d["name"] for d in x] if isinstance(x, list) else [])


In [300]:
# keywords["keywords_str"] = keywords["keywords_list"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")


In [292]:
# from sentence_transformers import SentenceTransformer
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # Load a pre-trained sentence embedding model
# model = SentenceTransformer("all-MiniLM-L6-v2")

# # Generate embeddings
# keywords["embedding"] = keywords["keywords_str"].apply(lambda x: model.encode(x))

In [293]:
# # Convert to NumPy array
# embedding_matrix = np.vstack(df["embedding"].values)

# # Compute cosine similarity
# similarity_matrix = cosine_similarity(embedding_matrix, embedding_matrix)

# # Convert to DataFrame
# similarity_df = pd.DataFrame(similarity_matrix, index=df["movieId"], columns=df["movieId"])

# # Find most similar movies
# similar_movies = similarity_df[1].sort_values(ascending=False).head(10)

# Create user x movie matrix on ratings

In [295]:
from scipy.sparse import csr_matrix
import numpy as np

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.
    
    Args:
        df: pandas dataframe containing 3 columns (userId, movieId, rating)
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """

    # Number of unique users and movies to form matrix
    M = df['userId'].nunique()
    N = df['movieId'].nunique()

    # Map user/movie ID to an index (id, index)
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    
    # Map an index to a user/movie ID (index,id)
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    
    # list of unique indexes for users and movies
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]

    
    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

In [38]:
X.shape

(270896, 45115)

In [301]:
n_total = X.shape[0]*X.shape[1]     # number of all user movie combination
n_ratings = X.nnz                   # number of ratings
sparsity = n_ratings/n_total
print(f"Matrix sparsity: {round(sparsity*100,2)}%")

Matrix sparsity: 0.21%


# item-item recommendations with k-nn
**What movies were rated similarly by users?**

In [302]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given movie id.
    
    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations
    
    Output: returns list of k similar movie ID's
    """
    X = X.T
    neighbour_ids = []
    
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [303]:
search = 1
similar_movies = find_similar_movies(search, X, movie_mapper, movie_inv_mapper, k=10)
similar_movies

[260, 780, 1270, 3114, 480, 356, 648, 1210, 364]

In [304]:
merged_df[merged_df['movieId'] == search]['title'][0]

'Toy Story'

In [305]:
merged_df[merged_df['movieId'].isin(similar_movies)]['title']

256               Star Wars
351            Forrest Gump
359           The Lion King
475           Jurassic Park
638     Mission: Impossible
761        Independence Day
1170     Return of the Jedi
1228     Back to the Future
3002            Toy Story 2
Name: title, dtype: object

In [119]:
merged_df[merged_df['movieId'].isin(similar_movies)][['belongs_to_collection','genres','original_title','popularity','title','vote_average','vote_count','movieId','imdbId','tmdbId']]

,belongs_to_collection,genres,original_title,popularity,title,vote_average,vote_count,movieId,imdbId,tmdbId
256,"{'id': 10, 'name': 'Star Wars Collection', 'po...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Star Wars,42.149697,Star Wars,8.1,6778.0,260,76759,11.0
351,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Forrest Gump,48.307194,Forrest Gump,8.2,8147.0,356,109830,13.0
359,"{'id': 94032, 'name': 'The Lion King Collectio...","[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",The Lion King,21.605761,The Lion King,8.0,5520.0,364,110357,8587.0
475,"{'id': 328, 'name': 'Jurassic Park Collection'...","[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",Jurassic Park,8.863776,Jurassic Park,7.6,4956.0,480,107290,329.0
638,"{'id': 87359, 'name': 'Mission: Impossible Col...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Mission: Impossible,15.793477,Mission: Impossible,6.7,2677.0,648,117060,954.0
761,"{'id': 304378, 'name': 'Independence Day Colle...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Independence Day,17.634197,Independence Day,6.7,3334.0,780,116629,602.0
1170,"{'id': 10, 'name': 'Star Wars Collection', 'po...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Return of the Jedi,14.586087,Return of the Jedi,7.9,4763.0,1210,86190,1892.0
1228,"{'id': 264, 'name': 'Back to the Future Collec...","[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",Back to the Future,25.778509,Back to the Future,8.0,6239.0,1270,88763,105.0
3002,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story 2,17.547693,Toy Story 2,7.3,3914.0,3114,120363,863.0


# Purely based on genre

In [306]:
import ast

# Convert string representation of list to actual list
merged_df["genres"] = merged_df["genres"].apply(ast.literal_eval)

# Extract only the 'name' values
merged_df["genres_list"] = merged_df["genres"].apply(lambda x: [d["name"] for d in x] if isinstance(x, list) else [])

# Verify the transformation
print(merged_df["genres_list"].head())


0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres_list, dtype: object


In [308]:
genres = set(g for G in merged_df["genres_list"] for g in G)
test = merged_df[['movieId','genres_list']]

for g in genres:
    test[g] = test.genres_list.transform(lambda x: int(g in x))

C:\Users\JaeHoBahng\AppData\Local\Temp\ipykernel_8532\209495771.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[g] = test.genres_list.transform(lambda x: int(g in x))
C:\Users\JaeHoBahng\AppData\Local\Temp\ipykernel_8532\209495771.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[g] = test.genres_list.transform(lambda x: int(g in x))
C:\Users\JaeHoBahng\AppData\Local\Temp\ipykernel_8532\209495771.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [ ]:
test = test.reset_index(drop=True)
movie_mapper = dict(zip(list(range(test.shape[0])), test["movieId"]))
movie_genres = test.drop(columns=['genres_list','movieId'])

In [226]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(movie_genres, movie_genres)
print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")

Dimensions of our genres cosine similarity matrix: (45525, 45525)


In [269]:
idx=0
n_recommendations=30
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:(n_recommendations+1)]
similar_movies = [i[0] for i in sim_scores]

In [270]:
genre_match = [movie_mapper[idx] for idx in similar_movies]

In [271]:
merged_df[merged_df['movieId'].isin(genre_match)]['title']

696                                      Oliver & Company
728                                         A Close Shave
1113                                   The Wrong Trousers
1722                                     Meet the Deedles
1943                            The Great Mouse Detective
2457                                     Doug's 1st Movie
3002                                          Toy Story 2
3314                                    Creature Comforts
3634                                          Chicken Run
4763                                       Monsters, Inc.
5636          The Looney, Looney, Looney Bugs Bunny Movie
6806                         Looney Tunes: Back in Action
7450     Bon Voyage, Charlie Brown (and Don't Come Back!)
7741                                             Garfield
8220                                          Bébé's Kids
8260                      The SpongeBob SquarePants Movie
9221                             An Extremely Goofy Movie
9368          

# Users with similar taste also watch

In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given movie id.
    
    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations
    
    Output: returns list of k similar movie ID's
    """
    X = X.T
    neighbour_ids = []
    
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [ ]:
user_inv_mapper

In [311]:
search = 1
similar_users = find_similar_movies(search, X.T, user_mapper, user_inv_mapper, k=10)
similar_users

[158371, 17415, 127566, 103723, 140899, 116913, 6354, 192355, 86271]

array([1. , 4.5, 5. , ..., 4.5, 4.5, 2. ])

# Currently Trending

# Best alltime

# Movies like this (Songs like this on spotify)
## What I'm used to
## Completely new